In [1]:
import cv2
import time
import pickle
import skimage.io
import numpy as np
import tkinter as tk
from functions import *
from tkinter import *
from matplotlib import pyplot as plt
from tkinter import filedialog as fd
from tkinter import messagebox as mb

def packPixels():
    overlapCounter = np.zeros((h,w),dtype='uint8')
    for idx in range(len(locs)):
        size = int(locs[idx][6])
        offset = size/2.0
        x , y = np.round(locs[idx][:2])
        x1,y1,x2,y2 = int(x-offset),int(y-offset),int(x+offset),int(y+offset)
        overlapCounter[y1:y2,x1:x2] += 1
    nLayers = np.max(overlapCounter)
    coeff_layers = []
    [coeff_layers.append(np.zeros((h,w))) for i in range(nLayers)]
    index_layers = []
    [index_layers.append(np.zeros((h,w))) for i in range(nLayers)]
    for idx in range(len(locs)):
        values = coeffs[idx]
        size = int(locs[idx][6])
        offset = size/2.0
        x , y = np.round(locs[idx][:2])
        pixelCoords = [(int(x+i-offset), int(y+j-offset), values[j,i]) for j in range(size) for i in range(size)]
        for coord in pixelCoords:
            a , b , c = coord
            for i in range(nLayers):
                if coeff_layers[i][b,a] == 0:
                    coeff_layers[i][b,a] = c
                    index_layers[i][b,a] = idx+1
                    break
    scaling_factor = ((2**coeff_bits)-1) / np.max(coeff_layers)
    for i in range(len(coeff_layers)):
        coeff_layers[i] = (coeff_layers[i]*scaling_factor).astype(numpy_type[types[COEFFICIENTS]])
        index_layers[i] = index_layers[i].astype(numpy_type[types[INDEX]])
    return coeff_layers,index_layers

def packKernels():
    pass

numpy_type = {
    'unsigned char': 'uint8',
    'unsigned short': 'uint16',
    'unsigned int': 'uint32',
    'unsigned long long': 'uint64'
}

coeffs = np.squeeze(pickle.load(open("8k_coeff.pkl", "rb" ), encoding='latin1'))
locs = pickle.load(open("8k_loc.pkl", "rb" ), encoding='latin1')
w = 2*int(np.max(np.abs(locs[:,0])) + np.max(locs[:,6]//2)+1)
h = 2*int(np.max(np.abs(locs[:,1])) + np.max(locs[:,6]//2)+1)
coeff_bits = 8

types = pickle.load(open("types.pkl", "rb"))

generateLayers = True
method = packPixels

if generateLayers:
    coeff_layers,index_layers = method()
    pickle.dump((coeff_layers,index_layers,len(locs)+1), open("layers.pkl", "wb"))
else:
    coeff_layers,index_layers, idx = pickle.load(open("layers.pkl", "rb"))

FileNotFoundError: [Errno 2] No such file or directory: 'types.pkl'